<a href="https://colab.research.google.com/github/vikrantpotnis123/DS/blob/master/ssd_train_eval_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -qq Cython contextlib2 pillow lxml matplotlib pandas tf-slim

Selecting previously unselected package python-bs4.
(Reading database ... 145480 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpacking pyt

In [4]:
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv

import re
import os
import io
import glob
import shutil
import urllib.request
import tarfile
import xml.etree.ElementTree as ET

import tensorflow.compat.v1 as tf

In [5]:
print(tf.__version__)

1.15.2


In [6]:
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv
import re
import cv2 
import os
import glob
import xml.etree.ElementTree as ET

import io
import tensorflow.compat.v1 as tf

from PIL import Image
from collections import namedtuple, OrderedDict

import shutil
import urllib.request
import tarfile

from google.colab import files

In [7]:
!nvidia-smi


Sun Jan  3 16:11:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
from google.colab import drive
 
drive.mount('/gdrive')# the project's folder

Mounted at /gdrive


In [9]:
%cd /gdrive/

/gdrive


In [10]:
%ls

MyDrive/  Shareddrives/


In [11]:
%cd MyDrive/

/gdrive/MyDrive


In [12]:
%cd ssd_object_detection/

/gdrive/MyDrive/ssd_object_detection


In [ ]:
!git clone --q https://github.com/tensorflow/models.git

In [13]:
%cd models/research/object_detection/

/gdrive/MyDrive/ssd_object_detection/models/research/object_detection


In [14]:
%cd ..

/gdrive/MyDrive/ssd_object_detection/models/research


In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [27]:
os.environ['PYTHONPATH'] += ':./:./slim/'

In [28]:
!echo $PYTHONPATH

/tensorflow-1.15.2/python3.6:/env/python:./:./slim/


In [ ]:
!python3 object_detection/builders/model_builder_tf1_test.py

Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_experimental_model
[       OK ] ModelBuilderTF1Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature1 (False)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_model_from_config_wi

In [16]:
tf.test.is_gpu_available()

True

In [17]:
%pwd

'/gdrive/MyDrive/ssd_object_detection/models/research'

In [18]:
%cd /gdrive/MyDrive/ssd_object_detection/pre_trained_models/

/gdrive/MyDrive/ssd_object_detection/pre_trained_models


In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz

--2021-01-03 00:41:14--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.20.128, 2607:f8b0:400e:c07::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.20.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187925923 (179M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_coco_2018_03_29.tar.gz’

ssd_mobilenet_v2_co 100%[===================>] 179.22M  67.0MB/s    in 2.7s    

2021-01-03 00:41:17 (67.0 MB/s) - ‘ssd_mobilenet_v2_coco_2018_03_29.tar.gz’ saved [187925923/187925923]



In [ ]:
!tar xvfz ssd_mobilenet_v2_coco_2018_03_29.tar.gz

ssd_mobilenet_v2_coco_2018_03_29/checkpoint
ssd_mobilenet_v2_coco_2018_03_29/model.ckpt.meta
ssd_mobilenet_v2_coco_2018_03_29/pipeline.config
ssd_mobilenet_v2_coco_2018_03_29/saved_model/saved_model.pb
ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb
ssd_mobilenet_v2_coco_2018_03_29/saved_model/
ssd_mobilenet_v2_coco_2018_03_29/saved_model/variables/
ssd_mobilenet_v2_coco_2018_03_29/model.ckpt.index
ssd_mobilenet_v2_coco_2018_03_29/
ssd_mobilenet_v2_coco_2018_03_29/model.ckpt.data-00000-of-00001


In [19]:
%pwd

'/gdrive/MyDrive/ssd_object_detection/pre_trained_models'

In [20]:
%cd ..

/gdrive/MyDrive/ssd_object_detection


In [21]:
%ls


models/                             pre_trained_models/
pipeline_mobilenet_v2_colab.config  vikrant_train/


In [ ]:
%pwd

'/gdrive/My Drive/ssd_object_detection'

In [24]:
%cd models/research

/gdrive/My Drive/ssd_object_detection/models/research


In [25]:
%ls /gdrive/MyDrive/ssd_object_detection/pipeline_mobilenet_v2_colab.config

/gdrive/MyDrive/ssd_object_detection/pipeline_mobilenet_v2_colab.config


In [29]:
!echo $PYTHONPATH


/tensorflow-1.15.2/python3.6:/env/python:./:./slim/


In [30]:
%ls

a3c_blogpost/        cvt_text/       lstm_object_detection/  rebar/
adversarial_text/    deeplab/        marco/                  seq_flow_lite/
attention_ocr/       deep_speech/    nst_blogpost/           setup.py
audioset/            delf/           object_detection/       slim/
autoaugment/         efficient-hrl/  pcl_rl/                 vid2depth/
cognitive_planning/  lfads/          README.md


In [31]:
!pip install lvis

In [ ]:
!python3 object_detection/model_main.py --model_dir='/gdrive/MyDrive/ssd_object_detection/vikrant_train/training_output' --pipeline_config_path='/gdrive/MyDrive/ssd_object_detection/pipeline_mobilenet_v2_colab.config'

In [ ]:
!python3 object_detection/export_tflite_ssd_graph.py --input_type=image_tensor --pipeline_config_path='/gdrive/MyDrive/ssd_object_detection/pipeline_mobilenet_v2_colab.config' --output_directory='/gdrive/MyDrive/ssd_object_detection/output_dir' --trained_checkpoint_prefix='/gdrive/MyDrive/ssd_object_detection/vikrant_train/training_output/model.ckpt-50479'

Instructions for updating:
Please use `layer.__call__` method instead.
W0103 16:52:58.770142 139946398910336 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0103 16:53:01.044320 139946398910336 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0103 16:53:01.079082 139946398910336 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0103 16:53:01.111297 139946398910336 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

In [39]:
!python3 object_detection/export_inference_graph.py --input_type=image_tensor --pipeline_config_path='/gdrive/MyDrive/ssd_object_detection/pipeline_mobilenet_v2_colab.config' --output_directory='/gdrive/MyDrive/ssd_object_detection/output_dir' --trained_checkpoint_prefix='/gdrive/MyDrive/ssd_object_detection/vikrant_train/training_output/model.ckpt-50479'

Instructions for updating:
Please use `layer.__call__` method instead.
W0103 17:01:25.960448 140294544373632 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0103 17:01:28.202522 140294544373632 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0103 17:01:28.242439 140294544373632 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0103 17:01:28.282015 140294544373632 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

In [41]:
%ls '/gdrive/MyDrive/ssd_object_detection/output_dir'

checkpoint                      model.ckpt.index  saved_model/
frozen_inference_graph.pb       model.ckpt.meta   tflite_graph.pb
model.ckpt.data-00000-of-00001  pipeline.config   tflite_graph.pbtxt
